# Baseline Models

In [1]:
import numpy as np

import gensim

import math

from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import GRU, LSTM, Activation, Dense, Dropout, Embedding
from tensorflow.keras.callbacks import History

D:\giuse\Conda\envs\thesis\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
w2v_model = gensim.models.KeyedVectors.load("w2v.model", mmap='r')

In [3]:
vocab_size, emdedding_size = w2v_model.wv.vectors.shape
vocab_size, emdedding_size

(28674, 128)

In [4]:
x = np.load('data/x.npy')
y = np.load('data/y.npy')[:,0]

In [5]:
x.shape, y.shape

((11029, 159), (11029,))

In [6]:
# instantiate history to save losses
history = History()

## Baseline #1: GRU

In [7]:
gru = Sequential()

gru.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size))
gru.add(GRU(256, input_shape=(vocab_size, emdedding_size), return_sequences=True))
gru.add(Dropout(0.3))
gru.add(GRU(256, return_sequences=True))
gru.add(Dropout(0.3))
gru.add(GRU(128))
gru.add(Dropout(0.3))
gru.add(Dense(vocab_size, activation='softmax'))

In [8]:
gru.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [9]:
filepath = "weights/gru.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

In [ ]:
gru_loss = gru.fit(x, y, validation_split=0.2, batch_size=64, epochs=20, callbacks=callbacks)

Train on 8823 samples, validate on 2206 samples
Epoch 1/20
8768/8823 [============================>.] - ETA: 2s - loss: 7.7236
Epoch 00001: val_loss improved from inf to 6.97640, saving model to weights/gru.hdf5
8823/8823 [==============================] - 434s 49ms/sample - loss: 7.7137 - val_loss: 6.9764
Epoch 2/20
8768/8823 [============================>.] - ETA: 2s - loss: 6.5794
Epoch 00002: val_loss improved from 6.97640 to 6.74908, saving model to weights/gru.hdf5
8823/8823 [==============================] - 450s 51ms/sample - loss: 6.5775 - val_loss: 6.7491
Epoch 3/20
8768/8823 [============================>.] - ETA: 2s - loss: 6.0505
Epoch 00003: val_loss improved from 6.74908 to 6.28922, saving model to weights/gru.hdf5
8823/8823 [==============================] - 480s 54ms/sample - loss: 6.0517 - val_loss: 6.2892
Epoch 4/20
8768/8823 [============================>.] - ETA: 2s - loss: 5.5431
Epoch 00004: val_loss improved from 6.28922 to 6.04134, saving model to weights/gru.h

## Baseline #2: GRU + Word2Vec

In [ ]:
gru_w2v = Sequential()

gru_w2v.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[w2v_model.wv.vectors]))
gru_w2v.add(GRU(256, input_shape=(vocab_size, emdedding_size), return_sequences=True))
gru_w2v.add(Dropout(0.2))
gru_w2v.add(GRU(256, return_sequences=True))
gru_w2v.add(Dropout(0.2))
gru_w2v.add(GRU(128))
gru_w2v.add(Dropout(0.2))
gru_w2v.add(Dense(vocab_size, activation='softmax'))

In [ ]:
gru_w2v.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [ ]:
filepath = "weights/gru_w2v.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

In [ ]:
gru_w2v_loss = gru_w2v.fit(x, y, validation_split=0.2, batch_size=64, epochs=20, callbacks=callbacks)

## Baseline #3: LSTM 

In [ ]:
lstm = Sequential()

lstm.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size))
lstm.add(LSTM(256, input_shape=(vocab_size, emdedding_size), return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(256, return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(256))
lstm.add(Dropout(0.2))
lstm.add(Dense(vocab_size, activation='softmax'))

In [ ]:
lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [ ]:
filepath = "weights/lstm.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

In [ ]:
lstm_loss = lstm.fit(x, y, validation_split=0.2, batch_size=64, epochs=20, callbacks=callbacks)

## Baseline #4: LSTM + Word2Vec 

In [ ]:
lstm_w2v = Sequential()

lstm_w2v.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[w2v_model.wv.vectors]))
lstm_w2v.add(LSTM(256, input_shape=(vocab_size, emdedding_size), return_sequences=True))
lstm_w2v.add(Dropout(0.2))
lstm_w2v.add(LSTM(256, return_sequences=True))
lstm_w2v.add(Dropout(0.2))
lstm_w2v.add(LSTM(128))
lstm_w2v.add(Dropout(0.2))
lstm_w2v.add(Dense(vocab_size, activation='softmax'))

In [ ]:
lstm_w2v.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [ ]:
filepath = "weights/lstm_w2v.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

In [ ]:
lstm_w2v_loss = lstm_w2v.fit(x, y, validation_split=0.2, batch_size=64, epochs=20, callbacks=callbacks)

## Generation

In [ ]:
def sample(preds, temperature):
    if temperature <= 0:
        return np.argmax(preds)

    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
  
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
  
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def word_to_id(word):
    return w2v_model.wv.key_to_index[word]

def id_to_word(id):
    return w2v_model.wv.index_to_key[id]

In [ ]:
def generate(model=gru, prompt='In this paper', n=20, temperature=0.2):
    word_ids = [word_to_id(word) for word in prompt.lower().split()]
    
    for i in range(n):
        prediction = model.predict(x=np.array(word_ids))
        id = sample(prediction[-1], temperature)
        word_ids.append(id)
    
    return ' '.join(id_to_word(id) for id in word_ids)

In [ ]:
generate()

In [ ]:
generate(model=lstm_w2v, n=50)

In [ ]:
generate(prompt='In this paper we present a novel approach', n=100)

## Evaluation

In [ ]:
models = {gru_loss: 'GRU', gru_w2v_loss: 'GRU + Word2Vec', lstm_loss: 'LSTM', lstm_w2v_loss: 'LSTM + Word2Vec'}

In [ ]:
# get minimimum validation loss within a set num of epochs
def min_val_loss(model, max_epochs=50):
    return min(model.history['val_loss'][:max_epochs])

In [ ]:
for m in models.keys():
    print("Minimum validation loss for {}: {:.5f}".format(models[m], min_val_loss(m)))
    print("Perplexity for model {}: {:.2f}\n".format(models[m], math.exp(min_val_loss(m))))